In [80]:
!pip install tensorflow

In [81]:
import os # операционная система (работа с папкой)
import requests # запросы по url
import re # регулярные выражения
from bs4 import BeautifulSoup # разобрать html
import io # работа с файлами (загрузка / выгрузка)
import shutil # переместить из папки в папку

# from google.colab import files # скачать из colab

import pandas as pd

# Классификация нового изображения
from PIL import Image
from torchvision import transforms
import torch

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD

from sklearn.model_selection import train_test_split

In [36]:
# Загрузка данных
GENERAL_URL = 'https://yandex.ru/images/search?text={query_text}&p={page}'
DIR_NAME_class01 = 'пиджак'
DIR_NAME_class02 = 'брюки'
ENCODING = 'utf-8'
MOZILLA_SEARCH_HEADER = {'User-Agent':'Mozilla/5.0'} # Браузер Мозилла
HTML_MODE = 'html.parser'
HREF_MODE = True # включить запись ссылок

In [4]:
# Функция парсера URL
def parser_url(url):
  pattern = r'img_url=([^&]+)&text='
  match_u = re.search(pattern, url)
  if match_u:
    img_url_encoded = match_u.group(1)
    img_url_decoded = img_url_encoded.replace('%2F', '/').replace('%3A', ':')
    print(img_url_decoded)
    return img_url_decoded
  else:
    print('Ссылка после img_url не найдена в URL')

In [5]:
# Функция загрузки 1 изображения
def download_image(url, save_path):
  try:
    response = requests.get(url, headers=MOZILLA_SEARCH_HEADER, stream=True)
    if(response.status_code == 200):
      with open(save_path, 'wb') as file:
        for chunk in response.iter_content(1024):
          file.write(chunk)
      return True
    else:
      print(f'Не удалось загрузить изображение: {url}')
      return False
  except Exception as e:
    print(f'Ошибка при загрузке изображения: {url}')
    return False

In [6]:
def get_links_on_keyword(keyword, page_num=1, encoding=ENCODING):
  '''
  Взять по ключевому слову набор ссылок с сайта ХабраХабр
  @param keyword - ключевое слово
  @param page_num - сколько страниц смотрим (как правило, на 1 странице 20 статей)
  '''
  links_to_find = []
  names_to_find = []
  __PARSE_TAG = 'a'
  __PARSE_CLASS = 'serp-item__link'

  total_len = page_num
  keyword_lwr = keyword.lower()
  downloaded_count = 0

  for page in range(page_num + 1):
    page_articles = GENERAL_URL.format(page=page_num, query_text=keyword)
    response = requests.get(page_articles, headers=MOZILLA_SEARCH_HEADER) # Браузер Мозилла
    response.encoding = encoding
    page_data_soup = BeautifulSoup(response.text, HTML_MODE)
    if(response.status_code == 200):
      for link in page_data_soup.find_all(__PARSE_TAG, HREF_MODE, class_=__PARSE_CLASS):
        print(link['href'])
        # получаем полный URL изображения
        img_url = parser_url(link['href'])
        print(img_url)
        links_to_find.append[img_url]
  return names_to_find, links_to_find

In [7]:
# Функция проверки директории
def check_repo_dataset(class_name, current_dir="./"):
  class_folder = os.path.join(current_dir + '/datasets', class_name)
  if not os.path.exists(class_folder):
    os.makedirs(class_folder)
    return class_folder
  else:
    return class_folder

In [8]:
# Функция колличество изображений в выдаче
def calc_pages(num_images):
  return num_images // 5 + (num_images % 5 > 0) if num_images > 5 else 1

In [9]:
# Функция загрузки изображений
def download_images(query, num_images, mini_images = False):
  pages = calc_pages(num_images)
  class_folder = check_repo_dataset(query, "./")
  downloaded_count = 0
  base_url = 'https:'
  # а вот это чтобы без движков было, грузим странички
  for page in range(0, pages):
    search_url = GENERAL_URL.format(query_text=query, page=page)
    response = requests.get(search_url, headers=MOZILLA_SEARCH_HEADER)
    soup = BeautifulSoup(response.text, 'html.parser')
    if (mini_images == False):
      for a in soup.find_all('a', class_='serp-item__link'):
        img_url = a['href']
        # получаем полный URL изображения
        img_url = parser_url(img_url)
        image_filename = f'{downloaded_count:04d}.jpg'
        image_path = os.path.join(class_folder, image_filename)
        if(download_image(img_url, image_path)):
          downloaded_count += 1
          print(f"Загружено изображений для {query}: {downloaded_count}/{num_images}")
        if(downloaded_count >= num_images):
          break
    else:
      for a in soup.find_all('img', class_='serp-item__thumb'):
        img_url = a['src']
        # из за получение //avatar, надо бы добавить https:// чтобы ссылка стала полной
        if(not img_url.startswith('http')):
          img_url = base_url + img_url
          image_filename = f'{downloaded_count:04d}.jpg'
          image_path = os.path.join(class_folder, image_filename)
          if(download_image(img_url, image_path)):
            downloaded_count += 1
            print(f'Загружено изображений для {query}: {downloaded_count}/{num_images}')
          if(downloaded_count >= num_images):
            break

In [10]:
# Загрузка 1 набора изображений
download_images(DIR_NAME_class01, num_images=80, mini_images=True)

Загружено изображений для пиджак: 1/200
Загружено изображений для пиджак: 2/200
Загружено изображений для пиджак: 3/200
Загружено изображений для пиджак: 4/200
Загружено изображений для пиджак: 5/200
Загружено изображений для пиджак: 6/200
Загружено изображений для пиджак: 7/200
Загружено изображений для пиджак: 8/200
Загружено изображений для пиджак: 9/200
Загружено изображений для пиджак: 10/200
Загружено изображений для пиджак: 11/200
Загружено изображений для пиджак: 12/200
Загружено изображений для пиджак: 13/200
Загружено изображений для пиджак: 14/200
Загружено изображений для пиджак: 15/200
Загружено изображений для пиджак: 16/200
Загружено изображений для пиджак: 17/200
Загружено изображений для пиджак: 18/200
Загружено изображений для пиджак: 19/200
Загружено изображений для пиджак: 20/200
Загружено изображений для пиджак: 21/200
Загружено изображений для пиджак: 22/200
Загружено изображений для пиджак: 23/200
Загружено изображений для пиджак: 24/200
Загружено изображений для

In [39]:
# Загрузка 2 набора изображений
download_images(DIR_NAME_class02, num_images=80, mini_images=True)

Загружено изображений для брюки: 1/200
Загружено изображений для брюки: 2/200
Загружено изображений для брюки: 3/200
Загружено изображений для брюки: 4/200
Загружено изображений для брюки: 5/200
Загружено изображений для брюки: 6/200
Загружено изображений для брюки: 7/200
Загружено изображений для брюки: 8/200
Загружено изображений для брюки: 9/200
Загружено изображений для брюки: 10/200
Загружено изображений для брюки: 11/200
Загружено изображений для брюки: 12/200
Загружено изображений для брюки: 13/200
Загружено изображений для брюки: 14/200
Загружено изображений для брюки: 15/200
Загружено изображений для брюки: 16/200
Загружено изображений для брюки: 17/200
Загружено изображений для брюки: 18/200
Загружено изображений для брюки: 19/200
Загружено изображений для брюки: 20/200
Загружено изображений для брюки: 21/200
Загружено изображений для брюки: 22/200
Загружено изображений для брюки: 23/200
Загружено изображений для брюки: 24/200
Загружено изображений для брюки: 25/200
Загружено

In [33]:
!zip -r '/content/{DIR_NAME_class01}.zip' '/content/datasets/{DIR_NAME_class01}'

  adding: content/datasets/пиджак/ (stored 0%)
  adding: content/datasets/пиджак/0123.jpg (deflated 2%)
  adding: content/datasets/пиджак/0023.jpg (deflated 0%)
  adding: content/datasets/пиджак/0033.jpg (deflated 3%)
  adding: content/datasets/пиджак/0078.jpg (deflated 1%)
  adding: content/datasets/пиджак/0032.jpg (deflated 0%)
  adding: content/datasets/пиджак/0138.jpg (deflated 1%)
  adding: content/datasets/пиджак/0119.jpg (deflated 1%)
  adding: content/datasets/пиджак/0057.jpg (deflated 1%)
  adding: content/datasets/пиджак/0013.jpg (deflated 0%)
  adding: content/datasets/пиджак/0140.jpg (deflated 2%)
  adding: content/datasets/пиджак/0070.jpg (deflated 2%)
  adding: content/datasets/пиджак/0030.jpg (deflated 2%)
  adding: content/datasets/пиджак/0026.jpg (deflated 1%)
  adding: content/datasets/пиджак/0093.jpg (deflated 0%)
  adding: content/datasets/пиджак/0128.jpg (deflated 2%)
  adding: content/datasets/пиджак/0108.jpg (deflated 0%)
  adding: content/datasets/пиджак/0107.jp

In [40]:
!zip -r '/content/{DIR_NAME_class02}.zip' '/content/datasets/{DIR_NAME_class02}'

  adding: content/datasets/брюки/ (stored 0%)
  adding: content/datasets/брюки/0023.jpg (deflated 4%)
  adding: content/datasets/брюки/0033.jpg (deflated 0%)
  adding: content/datasets/брюки/0078.jpg (deflated 1%)
  adding: content/datasets/брюки/0032.jpg (deflated 1%)
  adding: content/datasets/брюки/0119.jpg (deflated 1%)
  adding: content/datasets/брюки/0057.jpg (deflated 0%)
  adding: content/datasets/брюки/0013.jpg (deflated 0%)
  adding: content/datasets/брюки/0070.jpg (deflated 0%)
  adding: content/datasets/брюки/0030.jpg (deflated 1%)
  adding: content/datasets/брюки/0026.jpg (deflated 1%)
  adding: content/datasets/брюки/0093.jpg (deflated 0%)
  adding: content/datasets/брюки/0108.jpg (deflated 3%)
  adding: content/datasets/брюки/0107.jpg (deflated 0%)
  adding: content/datasets/брюки/0081.jpg (deflated 0%)
  adding: content/datasets/брюки/0069.jpg (deflated 3%)
  adding: content/datasets/брюки/0051.jpg (deflated 0%)
  adding: content/datasets/брюки/0025.jpg (deflated 0%)
  

In [ ]:
# Распаковка архива датасета
# !unzip -q '/content/{DIR_NAME_class01}.zip'
# !unzip -q '/content/{DIR_NAME_class02}.zip'

In [41]:
# Получить файлы
folder = '/content/datasets/'
lst_files_01 = os.listdir(f'{folder}/{DIR_NAME_class01}')[:50] # 1. Считываем из папки; 2. Обрезаем датасет, 1
lst_files_02 = os.listdir(f'{folder}/{DIR_NAME_class02}')[:50] # 1. Считываем из папки; 2. Обрезаем датасет, 0

In [42]:
print(len(lst_files_01))
print(len(lst_files_02))

50
50


In [43]:
# Изменить имена файлов
image_file_path_01 = [f"{DIR_NAME_class01}_{file_name}" for file_name in lst_files_01]
image_file_path_02 = [f"{DIR_NAME_class02}_{file_name}" for file_name in lst_files_02]

In [46]:
# Собрать в датафрейм
df_class_01 = pd.DataFrame({'image_file_path': image_file_path_01, 'image_real_path': lst_files_01, 'target': 1})
df_class_02 = pd.DataFrame({'image_file_path': image_file_path_02, 'image_real_path': lst_files_02, 'target': 0})

df = pd.concat([df_class_01, df_class_02], ignore_index=True) # сложили датафрейм
df

,image_file_path,image_real_path,target
0,пиджак_0123.jpg,0123.jpg,1
1,пиджак_0023.jpg,0023.jpg,1
2,пиджак_0033.jpg,0033.jpg,1
3,пиджак_0078.jpg,0078.jpg,1
4,пиджак_0032.jpg,0032.jpg,1
...,...,...,...
95,брюки_0118.jpg,0118.jpg,0
96,брюки_0100.jpg,0100.jpg,0
97,брюки_0022.jpg,0022.jpg,0
98,брюки_0031.jpg,0031.jpg,0


In [53]:
# Для первого класса
X_train, X_test, y_train, y_test = train_test_split(df[['image_file_path', 'image_real_path']], df['target'], test_size=0.2, random_state=1234, stratify=df['target'])
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1234, stratify=y_test)

X_train

,image_file_path,image_real_path
0,пиджак_0123.jpg,0123.jpg
71,брюки_0114.jpg,0114.jpg
82,брюки_0106.jpg,0106.jpg
27,пиджак_0045.jpg,0045.jpg
81,брюки_0074.jpg,0074.jpg
...,...,...
85,брюки_0084.jpg,0084.jpg
93,брюки_0028.jpg,0028.jpg
15,пиджак_0108.jpg,0108.jpg
60,брюки_0093.jpg,0093.jpg


In [71]:
# Длина тренировочной выборки для первого класса
print('Длина тренировочной выборки: ' + str(len(y_train[y_train == 0])))

# Длина тестовой выборки для первого класса
print('Длина тестовой выборки: ' + str(len(y_test[y_test == 0])))

# Длина валидационной выборки для первого класса
print('Длина валидационной выборки: ' + str(len(y_val[y_val == 0])))

Длина тренировочной выборки: 40
Длина тестовой выборки: 5
Длина валидационной выборки: 5


In [56]:
# Добавляем столбец с классом для тренировочного датасета
X_train_full = X_train.copy()
X_train_full['target'] = y_train

X_train_full

,image_file_path,image_real_path,target
0,пиджак_0123.jpg,0123.jpg,1
71,брюки_0114.jpg,0114.jpg,0
82,брюки_0106.jpg,0106.jpg,0
27,пиджак_0045.jpg,0045.jpg,1
81,брюки_0074.jpg,0074.jpg,0
...,...,...,...
85,брюки_0084.jpg,0084.jpg,0
93,брюки_0028.jpg,0028.jpg,0
15,пиджак_0108.jpg,0108.jpg,1
60,брюки_0093.jpg,0093.jpg,0


In [66]:
# Добавляем столбец с классом для тестового датасета
X_test_full = X_test.copy()
X_test_full['target'] = y_test

X_test_full

,image_file_path,image_real_path,target
8,пиджак_0013.jpg,0013.jpg,1
80,брюки_0007.jpg,0007.jpg,0
19,пиджак_0121.jpg,0121.jpg,1
26,пиджак_0038.jpg,0038.jpg,1
25,пиджак_0148.jpg,0148.jpg,1
74,брюки_0000.jpg,0000.jpg,0
76,брюки_0062.jpg,0062.jpg,0
97,брюки_0022.jpg,0022.jpg,0
62,брюки_0107.jpg,0107.jpg,0
6,пиджак_0119.jpg,0119.jpg,1


In [60]:
# Добавляем столбец с классом для валидационного датасета
X_val_full = X_val.copy()
X_val_full['target'] = y_val

X_val_full

,image_file_path,image_real_path,target
91,брюки_0049.jpg,0049.jpg,0
20,пиджак_0069.jpg,0069.jpg,1
41,пиджак_0126.jpg,0126.jpg,1
65,брюки_0051.jpg,0051.jpg,0
16,пиджак_0107.jpg,0107.jpg,1
22,пиджак_0025.jpg,0025.jpg,1
73,брюки_0076.jpg,0076.jpg,0
88,брюки_0004.jpg,0004.jpg,0
69,брюки_0038.jpg,0038.jpg,0
45,пиджак_0102.jpg,0102.jpg,1


In [57]:
# Создаем необходимые директории
!mkdir "train_data"
!mkdir "test_data"
!mkdir "validation_data"

!mkdir '/content/train_data/{DIR_NAME_class01}/'
!mkdir '/content/train_data/{DIR_NAME_class02}/'

!mkdir '/content/test_data/{DIR_NAME_class01}/'
!mkdir '/content/test_data/{DIR_NAME_class02}/'

!mkdir '/content/validation_data/{DIR_NAME_class01}/'
!mkdir '/content/validation_data/{DIR_NAME_class02}/'

In [63]:
# Функция перемещения из папки в папку
def move_img_to_folder(path, path_to_move, url_file):
  shutil.move(f"{path}/{url_file}", path_to_move)

In [73]:
# Перемещаем данные
for index, row in X_train_full.iterrows():
  folder_from = f'/content/datasets/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/datasets/{DIR_NAME_class02}'
  folder_to_move = f'/content/train_data/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/train_data/{DIR_NAME_class02}'
  move_img_to_folder(folder_from, folder_to_move, row['image_real_path'])

for index, row in X_test_full.iterrows():
  folder_from = f'/content/datasets/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/datasets/{DIR_NAME_class02}'
  folder_to_move = f'/content/test_data/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/test_data/{DIR_NAME_class02}'
  move_img_to_folder(folder_from, folder_to_move, row['image_real_path'])

for index, row in X_val_full.iterrows():
  folder_from = f'/content/datasets/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/datasets/{DIR_NAME_class02}'
  folder_to_move = f'/content/validation_data/{DIR_NAME_class01}/' if row['target'] == 1 else f'/content/validation_data/{DIR_NAME_class02}'
  move_img_to_folder(folder_from, folder_to_move, row['image_real_path'])

In [75]:
!zip -r '/content/train_data.zip' '/content/train_data/'
!zip -r '/content/test_data.zip' '/content/test_data/'
!zip -r '/content/validation_data.zip' '/content/validation_data/'

  adding: content/train_data/ (stored 0%)
  adding: content/train_data/пиджак/ (stored 0%)
  adding: content/train_data/пиджак/0123.jpg (deflated 2%)
  adding: content/train_data/пиджак/0023.jpg (deflated 0%)
  adding: content/train_data/пиджак/0033.jpg (deflated 3%)
  adding: content/train_data/пиджак/0078.jpg (deflated 1%)
  adding: content/train_data/пиджак/0032.jpg (deflated 0%)
  adding: content/train_data/пиджак/0138.jpg (deflated 1%)
  adding: content/train_data/пиджак/0057.jpg (deflated 1%)
  adding: content/train_data/пиджак/0140.jpg (deflated 2%)
  adding: content/train_data/пиджак/0070.jpg (deflated 2%)
  adding: content/train_data/пиджак/0030.jpg (deflated 2%)
  adding: content/train_data/пиджак/0026.jpg (deflated 1%)
  adding: content/train_data/пиджак/0093.jpg (deflated 0%)
  adding: content/train_data/пиджак/0128.jpg (deflated 2%)
  adding: content/train_data/пиджак/0108.jpg (deflated 0%)
  adding: content/train_data/пиджак/0081.jpg (deflated 1%)
  adding: content/train_

In [ ]:
def classify_image(image_path, device, model, class_names):
  image = Image.open(image_path)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
  ])
  image = preprocess(image).unsqueeze(0)  # Добавление размерности батча (batch dimension)
  image = image.to(device)

  model.eval()
  with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

  return class_names[predicted[0]]

In [89]:
# Генерация данных
# Задаём параметры для предобработки изображений
image_size = (224, 224) # сложности с обработкой, (28,28) - сжатие изображения, при сжатии остаются основные моменты, компьютер осталяет самое важное
batch_size = 40 # 16, 32 для текста, для изображений больше 32
data = '/content'

# Создаем генератоы данных для тренировочного, валидационного и тестового наборов
train_datagen = ImageDataGenerator(
  rescale=1.0 / 255.0,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
  os.path.join(data, 'train_data'),
  target_size=image_size,
  batch_size=batch_size,
  class_mode='binary'
) # 224*224

test_datagen = ImageDataGenerator(
  rescale=1.0 / 255.0
)

test_generator = test_datagen.flow_from_directory(
  os.path.join(data, 'test_data'),
  target_size=image_size,
  batch_size=1,
  class_mode=None,
  shuffle=False
) # 224*224

validation_datagen = ImageDataGenerator(
  rescale=1.0 / 255.0
)

validation_generator = validation_datagen.flow_from_directory(
  os.path.join(data, 'validation_data'),
  target_size=image_size,
  batch_size=batch_size,
  class_mode='binary'
) # 224*224

Found 80 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [86]:
# Создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

In [90]:
# Обучаем модель
num_epochs = 30 # Количество эпох
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/30
2/2 [==============================] - 13s 7s/step - loss: 0.6950 - accuracy: 0.5000 - val_loss: 0.6692 - val_accuracy: 0.5000
Epoch 2/30
2/2 [==============================] - 10s 5s/step - loss: 0.6735 - accuracy: 0.5625 - val_loss: 0.6157 - val_accuracy: 1.0000
Epoch 3/30
2/2 [==============================] - 11s 5s/step - loss: 0.6421 - accuracy: 0.7125 - val_loss: 0.5419 - val_accuracy: 1.0000
Epoch 4/30
2/2 [==============================] - 11s 4s/step - loss: 0.6103 - accuracy: 0.6625 - val_loss: 0.4266 - val_accuracy: 1.0000
Epoch 5/30
2/2 [==============================] - 12s 6s/step - loss: 0.5519 - accuracy: 0.7500 - val_loss: 0.2834 - val_accuracy: 0.9000
Epoch 6/30
2/2 [==============================] - 11s 6s/step - loss: 0.4733 - accuracy: 0.8000 - val_loss: 0.1556 - val_accuracy: 1.0000
Epoch 7/30
2/2 [==============================] - 10s 5s/step - loss: 0.4337 - accuracy: 0.8375 - val_loss: 0.0595 - val_accuracy: 1.0000
Epoch 8/30
2/2 [==================

In [91]:
# Сохранение модели
model.save('classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [108]:
# тестирование на демонстрационных данных

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

  # открываем файл для записи результатов
  with open(output_file, 'w') as result_file:

    # получаем список поддиректорий (car и not_car)
    subdirectories = os.listdir(directory_path)

    for subdirectory in subdirectories:
      subdirectory_path = os.path.join(directory_path, subdirectory)

      # проверяем, является ли поддиректория директорией
      if os.path.isdir(subdirectory_path):

        # получаем список файлов в поддиректории
        image_files = os.listdir(subdirectory_path)
        print(image_files)

        for image_file in image_files:

          # полный путь к изображению
          image_path = os.path.join(subdirectory_path, image_file)

          # загружаем и классифицируем изображение
          img = image.load_img(image_path, target_size=(224, 224))
          x = image.img_to_array(img)
          x = x / 255.0
          x = x.reshape((1, 224, 224, 3))
          pred = model.predict(x)

          # определяем метку класса
          if pred[0][0] > 0.5:
              class_label = DIR_NAME_class01
          else:
              class_label = 'НЕ ' + str(DIR_NAME_class01)

          # выводим результат в консоль
          print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
          print(f'Training Accuracy: {history.history["accuracy"][-1]}')
          print(f'Validation Accuracy: {history.history["val_accuracy"][-1]}\n')

          # записываем результаты в файл
          result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
          result_file.write(f'Training Accuracy: {history.history["accuracy"][-1]}\n')
          result_file.write(f'Validation Accuracy: {history.history["val_accuracy"][-1]}\n')


# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file = '/content/test_results.txt'

# вызываем функцию для классификации демонстрационных данных
classify_images_in_directory(demo_data_directory, output_file)

['0119.jpg', '0013.jpg', '0121.jpg', '0148.jpg', '0038.jpg']
1/1 [==============================] - 0s 74ms/step
Image: пиджак/0119.jpg, Prediction: пиджак

Training Accuracy: 0.875
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 88ms/step
Image: пиджак/0013.jpg, Prediction: пиджак

Training Accuracy: 0.875
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 77ms/step
Image: пиджак/0121.jpg, Prediction: пиджак

Training Accuracy: 0.875
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 86ms/step
Image: пиджак/0148.jpg, Prediction: НЕ пиджак

Training Accuracy: 0.875
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 96ms/step
Image: пиджак/0038.jpg, Prediction: пиджак

Training Accuracy: 0.875
Validation Accuracy: 0.8999999761581421

['0107.jpg', '0000.jpg', '0062.jpg', '0007.jpg', '0022.jpg']
1/1 [==============================] - 0s 74ms/step
Image: брюки/

In [113]:
# Полученные результаты тестирования модели на демонстрационных данных показывают, что точность модели хорошая.

# Проведем дополнительные эксперементы

# Эксперимент 1_1 Уменьшение количества эпох с 20 до 10

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# уменьшаем количество эпох с 20 до 10
num_epochs = 10

# обучаем модель
history_1_1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_1_1_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_1_1.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_1_1.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_1_1.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_1_1.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_1_1 = '/content/test_results_experiment_1_1.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_1_1)

Epoch 1/10
2/2 [==============================] - 13s 7s/step - loss: 0.7004 - accuracy: 0.3875 - val_loss: 0.6697 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 12s 6s/step - loss: 0.6727 - accuracy: 0.5000 - val_loss: 0.6520 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 11s 5s/step - loss: 0.6642 - accuracy: 0.5250 - val_loss: 0.5938 - val_accuracy: 0.9000
Epoch 4/10
2/2 [==============================] - 12s 5s/step - loss: 0.6415 - accuracy: 0.6750 - val_loss: 0.5392 - val_accuracy: 0.9000
Epoch 5/10
2/2 [==============================] - 11s 5s/step - loss: 0.5981 - accuracy: 0.8500 - val_loss: 0.4134 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 11s 6s/step - loss: 0.5308 - accuracy: 0.7125 - val_loss: 0.2636 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 11s 6s/step - loss: 0.4593 - accuracy: 0.7500 - val_loss: 0.2943 - val_accuracy: 0.8000
Epoch 8/10
2/2 [==================

In [114]:
# Эксперимент 1_2 Уменьшение количества эпох с 20 до 4

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# уменьшаем количество эпох с 20 до 4
num_epochs = 4

# обучаем модель
history_1_2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_1_2_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_1_2.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_1_2.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_1_2.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_1_2.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_1_2 = '/content/test_results_experiment_1_2.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_1_2)

Epoch 1/4
2/2 [==============================] - 12s 6s/step - loss: 0.6869 - accuracy: 0.4375 - val_loss: 0.6456 - val_accuracy: 0.5000
Epoch 2/4
2/2 [==============================] - 12s 5s/step - loss: 0.6689 - accuracy: 0.5125 - val_loss: 0.5963 - val_accuracy: 0.9000
Epoch 3/4
2/2 [==============================] - 12s 5s/step - loss: 0.6355 - accuracy: 0.6625 - val_loss: 0.5328 - val_accuracy: 0.9000
Epoch 4/4
1/1 [==============================] - 0s 129ms/step
Image: пиджак/0119.jpg, Prediction: пиджак

Training Accuracy: 0.7749999761581421
Validation Accuracy: 1.0

1/1 [==============================] - 0s 65ms/step
Image: пиджак/0013.jpg, Prediction: пиджак

Training Accuracy: 0.7749999761581421
Validation Accuracy: 1.0

1/1 [==============================] - 0s 69ms/step
Image: пиджак/0121.jpg, Prediction: пиджак

Training Accuracy: 0.7749999761581421
Validation Accuracy: 1.0

1/1 [==============================] - 0s 73ms/step
Image: пиджак/0148.jpg, Prediction: НЕ пиджак


In [115]:
# Уменьшение количества эпох приводит к ухудшению и !нестабильности! итоговых показателей модели

# Эксперимент 2_1 Изменение размера пакета = 16 (изначально Batch Size = 40)

batch_size = 16

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# обучаем модель
history_2_1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_2_1_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_2_1.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_2_1.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_2_1.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_2_1.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_2_1 = '/content/test_results_experiment_2_1.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_2_1)

Epoch 1/4
2/2 [==============================] - 12s 5s/step - loss: 0.6959 - accuracy: 0.4875 - val_loss: 0.6758 - val_accuracy: 0.5000
Epoch 2/4
2/2 [==============================] - 11s 5s/step - loss: 0.6776 - accuracy: 0.5500 - val_loss: 0.5865 - val_accuracy: 0.9000
Epoch 3/4
2/2 [==============================] - 11s 6s/step - loss: 0.6469 - accuracy: 0.7000 - val_loss: 0.4829 - val_accuracy: 1.0000
Epoch 4/4
1/1 [==============================] - 0s 146ms/step
Image: пиджак/0119.jpg, Prediction: пиджак

Training Accuracy: 0.6875
Validation Accuracy: 1.0

1/1 [==============================] - 0s 85ms/step
Image: пиджак/0013.jpg, Prediction: пиджак

Training Accuracy: 0.6875
Validation Accuracy: 1.0

1/1 [==============================] - 0s 77ms/step
Image: пиджак/0121.jpg, Prediction: пиджак

Training Accuracy: 0.6875
Validation Accuracy: 1.0

1/1 [==============================] - 0s 67ms/step
Image: пиджак/0148.jpg, Prediction: НЕ пиджак

Training Accuracy: 0.6875
Validatio

In [116]:
# Эксперимент 2_2 Изменение размера пакета = 32 (изначально Batch Size = 40)

batch_size = 32

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# обучение модели
history_2_2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_2_2_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_2_2.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_2_2.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_2_2.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_2_2.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_2_2 = '/content/test_results_experiment_2_2.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_2_2)

Epoch 1/4
2/2 [==============================] - 13s 5s/step - loss: 0.6939 - accuracy: 0.5375 - val_loss: 0.6229 - val_accuracy: 0.8000
Epoch 2/4
2/2 [==============================] - 11s 5s/step - loss: 0.6414 - accuracy: 0.7250 - val_loss: 0.5394 - val_accuracy: 0.9000
Epoch 3/4
2/2 [==============================] - 11s 6s/step - loss: 0.5789 - accuracy: 0.7750 - val_loss: 0.3902 - val_accuracy: 1.0000
Epoch 4/4
1/1 [==============================] - 0s 133ms/step
Image: пиджак/0119.jpg, Prediction: пиджак

Training Accuracy: 0.5
Validation Accuracy: 0.800000011920929

1/1 [==============================] - 0s 70ms/step
Image: пиджак/0013.jpg, Prediction: НЕ пиджак

Training Accuracy: 0.5
Validation Accuracy: 0.800000011920929

1/1 [==============================] - 0s 69ms/step
Image: пиджак/0121.jpg, Prediction: пиджак

Training Accuracy: 0.5
Validation Accuracy: 0.800000011920929

1/1 [==============================] - 0s 66ms/step
Image: пиджак/0148.jpg, Prediction: НЕ пиджак


In [117]:
# Изменение размера пакета не приводит к существенным изменениям итоговых показателей модели

# Эксперимент 3_1 Увеличение количества нейронов в слоях

# используем batch_size = 40
batch_size = 40

# создание модели
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(224, 224, 3))) # увеличиваем количество нейронов
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3))) # увеличиваем количество нейронов
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3))) # увеличиваем количество нейронов
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компилируем модель
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# обучаем модель
num_epochs = 20
history_3_1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_3_1_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_3_1.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_3_1.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_3_1.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_3_1.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_3_1 = '/content/test_results_experiment_3_1.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_3_1)

Epoch 1/20
2/2 [==============================] - 36s 21s/step - loss: 0.6868 - accuracy: 0.5375 - val_loss: 0.6784 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 30s 15s/step - loss: 0.6728 - accuracy: 0.5000 - val_loss: 0.5976 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - 28s 14s/step - loss: 0.6479 - accuracy: 0.6500 - val_loss: 0.4988 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - 29s 16s/step - loss: 0.6211 - accuracy: 0.6625 - val_loss: 0.3910 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 28s 14s/step - loss: 0.4995 - accuracy: 0.7750 - val_loss: 0.4288 - val_accuracy: 0.6000
Epoch 6/20
2/2 [==============================] - 28s 14s/step - loss: 0.5095 - accuracy: 0.7500 - val_loss: 0.1592 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 28s 15s/step - loss: 0.5059 - accuracy: 0.7750 - val_loss: 0.2017 - val_accuracy: 0.9000
Epoch 8/20
2/2 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 229ms/step
Image: пиджак/0119.jpg, Prediction: пиджак

Training Accuracy: 0.737500011920929
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 147ms/step
Image: пиджак/0013.jpg, Prediction: пиджак

Training Accuracy: 0.737500011920929
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 162ms/step
Image: пиджак/0121.jpg, Prediction: пиджак

Training Accuracy: 0.737500011920929
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 149ms/step
Image: пиджак/0148.jpg, Prediction: НЕ пиджак

Training Accuracy: 0.737500011920929
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 158ms/step
Image: пиджак/0038.jpg, Prediction: пиджак

Training Accuracy: 0.737500011920929
Validation Accuracy: 0.8999999761581421

1/1 [==============================] - 0s 160ms/step
Image: брюки/0107.jpg, Prediction: НЕ пиджак

Training Accuracy: 0.73

In [118]:
# Эксперимент 3_2 Уменьшение количества нейронов в слоях

# используем batch_size = 40
batch_size = 40

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128)) # уменьшим количество нейронов в полносвязном слое
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99),
    metrics=['accuracy']
)

# обучение модели
num_epochs = 20
history_3_2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_3_2_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_3_2.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_3_2.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_3_2.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_3_2.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_3_2 = '/content/test_results_experiment_3_2.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_3_2)

Epoch 1/20
2/2 [==============================] - 12s 4s/step - loss: 0.6925 - accuracy: 0.5375 - val_loss: 0.6794 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 10s 5s/step - loss: 0.6864 - accuracy: 0.5000 - val_loss: 0.6186 - val_accuracy: 0.6000
Epoch 3/20
2/2 [==============================] - 9s 4s/step - loss: 0.6487 - accuracy: 0.6000 - val_loss: 0.5765 - val_accuracy: 0.6000
Epoch 4/20
2/2 [==============================] - 10s 4s/step - loss: 0.6211 - accuracy: 0.6625 - val_loss: 0.4504 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 10s 4s/step - loss: 0.5740 - accuracy: 0.7250 - val_loss: 0.3291 - val_accuracy: 0.9000
Epoch 6/20
2/2 [==============================] - 11s 6s/step - loss: 0.5021 - accuracy: 0.8125 - val_loss: 0.2789 - val_accuracy: 0.9000
Epoch 7/20
2/2 [==============================] - 9s 4s/step - loss: 0.8382 - accuracy: 0.6500 - val_loss: 0.2957 - val_accuracy: 0.9000
Epoch 8/20
2/2 [====================

In [119]:
# Увеличение количества нейронов в скрытых слоях приводит к улучшению итоговых показателей модели
# Уменьшение количества нейронов в полносвязном слое приводит к улучшению итоговых показателей модели

# Эксперимент 4_1 Изменение скорости обучения = 0.001, (изначально learning_rate=0.01)

# используем batch_size = 40
batch_size = 40

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.001, momentum=0.99),
    metrics=['accuracy']
)

# обучаем модель
num_epochs = 20
history_4_1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_4_1_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_4_1.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_4_1.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_4_1.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_4_1.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_4_1 = '/content/test_results_experiment_4_1.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_4_1)

Epoch 1/20
2/2 [==============================] - 12s 5s/step - loss: 0.6879 - accuracy: 0.5500 - val_loss: 0.6890 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 11s 5s/step - loss: 0.6785 - accuracy: 0.5125 - val_loss: 0.6762 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 12s 6s/step - loss: 0.6744 - accuracy: 0.5000 - val_loss: 0.6638 - val_accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 11s 5s/step - loss: 0.6642 - accuracy: 0.5000 - val_loss: 0.6484 - val_accuracy: 0.5000
Epoch 5/20
2/2 [==============================] - 11s 5s/step - loss: 0.6624 - accuracy: 0.5000 - val_loss: 0.6274 - val_accuracy: 0.6000
Epoch 6/20
2/2 [==============================] - 12s 5s/step - loss: 0.6460 - accuracy: 0.5375 - val_loss: 0.5992 - val_accuracy: 0.7000
Epoch 7/20
2/2 [==============================] - 11s 6s/step - loss: 0.6307 - accuracy: 0.6250 - val_loss: 0.5716 - val_accuracy: 1.0000
Epoch 8/20
2/2 [==================

In [120]:
# Эксперимент 4_2 Изменение скорости обучения = 0.1, (изначально learning_rate=0.01)

# используем batch_size = 40
batch_size = 40

# создание модели
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.1, momentum=0.99),
    metrics=['accuracy']
)

# обучаем модель
num_epochs = 20
history_4_2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('experiment_4_2_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_4_2.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_4_2.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_4_2.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_4_2.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_experiment_4_2 = '/content/test_results_experiment_4_2.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_experiment_4_2)

Epoch 1/20
2/2 [==============================] - 13s 5s/step - loss: 0.6873 - accuracy: 0.5500 - val_loss: 0.6525 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 12s 5s/step - loss: 0.6766 - accuracy: 0.4875 - val_loss: 0.5518 - val_accuracy: 0.9000
Epoch 3/20
2/2 [==============================] - 11s 6s/step - loss: 0.6039 - accuracy: 0.7625 - val_loss: 2.7013 - val_accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 11s 4s/step - loss: 1.6883 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 5/20
2/2 [==============================] - 11s 5s/step - loss: 0.6939 - accuracy: 0.5125 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 6/20
2/2 [==============================] - 10s 5s/step - loss: 0.6975 - accuracy: 0.5000 - val_loss: 0.6808 - val_accuracy: 0.7000
Epoch 7/20
2/2 [==============================] - 10s 4s/step - loss: 0.6801 - accuracy: 0.5375 - val_loss: 0.6786 - val_accuracy: 0.5000
Epoch 8/20
2/2 [==================

In [ ]:
# Изменение скорости обучения = 0.001, относительно learning_rate = 0.1, демонстрирует улучшение показателей модели

# подбор best модели

# используем batch_size = 40
batch_size = 40

# создание модели
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(224, 224, 3))) # увеличим количество нейронов в скрытом слое (с 32 до 64)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3))) # увеличим количество нейронов с 64 до 128
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3))) # увеличим количество нейронов со 128 до 256
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256)) # уменьшим количество нейронов в полносвязном слое (с 512 до 256)
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# компиляция модели
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(learning_rate=0.01, momentum=0.99), # используем learning_rate=0.01
    metrics=['accuracy']
)

# обучаем модель
num_epochs = 50 # увеличим кол-во эпох (с 30 изначального варианта до 50)
history_best = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# сохраняем модель
model.save('best_model.h5')

# создаем функцию classify_image
def classify_images_in_directory(directory_path, output_file):

    # открываем файл для записи результатов
    with open(output_file, 'w') as result_file:

        # получаем список поддиректорий (car и not_car)
        subdirectories = os.listdir(directory_path)

        for subdirectory in subdirectories:
            subdirectory_path = os.path.join(directory_path, subdirectory)

            # проверяем, является ли поддиректория директорией
            if os.path.isdir(subdirectory_path):

                # получаем список файлов в поддиректории
                image_files = os.listdir(subdirectory_path)

                for image_file in image_files:

                    # полный путь к изображению
                    image_path = os.path.join(subdirectory_path, image_file)

                    # загружаем и классифицируем изображение
                    img = image.load_img(image_path, target_size=(224, 224))
                    x = image.img_to_array(img)
                    x = x / 255.0
                    x = x.reshape((1, 224, 224, 3))
                    pred = model.predict(x)

                    # определяем метку класса
                    if pred[0][0] > 0.5:
                        class_label = DIR_NAME_class01
                    else:
                        class_label = 'НЕ ' + str(DIR_NAME_class01)

                    # выводим результат в консоль
                    print(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    print(f'Training Accuracy: {history_best.history["accuracy"][-1]}')
                    print(f'Validation Accuracy: {history_best.history["val_accuracy"][-1]}\n')

                    # записываем результаты в файл
                    result_file.write(f'Image: {os.path.join(subdirectory, image_file)}, Prediction: {class_label}\n')
                    result_file.write(f'Training Accuracy: {history_best.history["accuracy"][-1]}\n')
                    result_file.write(f'Validation Accuracy: {history_best.history["val_accuracy"][-1]}\n')

# путь к директории с демонстрационными данными
demo_data_directory = f'/content/test_data/'

# путь к файлу, в который будут записаны результаты классификации
output_file_best = '/content/test_results_experiment_best.txt'

# вызываем функцию для классификации
classify_images_in_directory(demo_data_directory, output_file_best)

Epoch 1/50
2/2 [==============================] - 29s 13s/step - loss: 0.6951 - accuracy: 0.4625 - val_loss: 0.6657 - val_accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 27s 14s/step - loss: 0.6762 - accuracy: 0.6125 - val_loss: 0.5958 - val_accuracy: 0.9000
Epoch 3/50
2/2 [==============================] - 27s 14s/step - loss: 0.6351 - accuracy: 0.6125 - val_loss: 0.5006 - val_accuracy: 0.9000
Epoch 4/50
2/2 [==============================] - 26s 13s/step - loss: 0.6049 - accuracy: 0.6875 - val_loss: 0.3760 - val_accuracy: 1.0000
Epoch 5/50
2/2 [==============================] - 28s 15s/step - loss: 0.5276 - accuracy: 0.7750 - val_loss: 0.2231 - val_accuracy: 1.0000
Epoch 6/50
2/2 [==============================] - 26s 13s/step - loss: 0.5029 - accuracy: 0.7500 - val_loss: 0.6649 - val_accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 28s 13s/step - loss: 0.6086 - accuracy: 0.6875 - val_loss: 0.1029 - val_accuracy: 1.0000
Epoch 8/50
2/2 [===========

In [ ]:
# Таким образом, проведя несколько проверок параметров модели, были определены следующие оптимальные показатели для используемого датасета:
# количество эпох обучения = 50;
# batch_size = 40;
# увеличение количества нейронов в скрытых слоях и уменьшение количества нейронов в полносвязном слое;
# learning_rate=0.01.